<a href="https://colab.research.google.com/github/EVA4-RS-Group/Phase2/blob/master/Untitled21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import io

In [4]:

!unzip data.zip

Archive:  data.zip
   creating: data/
   creating: data/test/
   creating: data/test/data_test/
  inflating: data/test/data_test/desktop.ini  
  inflating: data/test/data_test/img_002.jpg  
  inflating: data/test/data_test/img_005.jpg  
  inflating: data/test/data_test/img_012.jpg  
  inflating: data/test/data_test/img_015.jpg  
  inflating: data/test/data_test/img_022.jpg  
  inflating: data/test/data_test/img_025.jpg  
  inflating: data/test/data_test/img_032.jpg  
  inflating: data/test/data_test/img_035.jpg  
  inflating: data/test/data_test/img_042.jpg  
  inflating: data/test/data_test/img_045.jpg  
  inflating: data/test/data_test/img_052.jpg  
  inflating: data/test/data_test/img_055.jpg  
  inflating: data/test/data_test/img_062.jpg  
  inflating: data/test/data_test/img_065.jpg  
  inflating: data/test/data_test/img_072.jpg  
  inflating: data/test/data_test/img_075.jpg  
  inflating: data/test/data_test/img_082.jpg  
  inflating: data/test/data_test/img_085.jpg  
  inflating

In [3]:
!mkdir samples

In [5]:
import os
import torch
import time
import numpy as np
import matplotlib.pyplot as plt

import torch.utils.data
from torch import nn, optim
from torch.autograd import Variable
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from torch.distributions.normal import Normal
from torch.distributions import kl_divergence


In [11]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        try:
            nn.init.xavier_uniform_(m.weight.data)
            m.bias.data.fill_(0)
        except AttributeError:
            print("Skipping initialization of ", classname)

In [12]:
class VAE(nn.Module):
    def __init__(self, input_dim, dim, z_dim):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(input_dim, dim, 4, 2, 1),
            nn.BatchNorm2d(dim),
            nn.ReLU(True),
            nn.Conv2d(dim, dim, 4, 2, 1),
            nn.BatchNorm2d(dim),
            nn.ReLU(True),
            nn.Conv2d(dim, dim, 5, 1, 0),
            nn.BatchNorm2d(dim),
            nn.ReLU(True),
            nn.Conv2d(dim, z_dim * 2, 3, 1, 0),
            nn.BatchNorm2d(z_dim * 2)
        )

        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(z_dim, dim, 3, 1, 0),
            nn.BatchNorm2d(dim),
            nn.ReLU(True),
            nn.ConvTranspose2d(dim, dim, 5, 1, 0),
            nn.BatchNorm2d(dim),
            nn.ReLU(True),
            nn.ConvTranspose2d(dim, dim, 4, 2, 1),
            nn.BatchNorm2d(dim),
            nn.ReLU(True),
            nn.ConvTranspose2d(dim, input_dim, 4, 2, 1),
            nn.Tanh()
        )

        self.apply(weights_init)

    def forward(self, x):
        mu, logvar = self.encoder(x).chunk(2, dim=1)

        q_z_x = Normal(mu, logvar.mul(.5).exp())
        p_z = Normal(torch.zeros_like(mu), torch.ones_like(logvar))
        kl_div = kl_divergence(q_z_x, p_z).sum(1).mean()

        x_tilde = self.decoder(q_z_x.rsample())
        return x_tilde, kl_div

In [13]:
DEVICE1 = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [47]:
print(DEVICE1)

cuda


In [14]:
INPUT_DIM = 3
DIM = 6
Z_DIM =  8
model = VAE(INPUT_DIM, DIM, Z_DIM)

In [15]:
DEVICE2 = torch.device("cpu")

In [16]:
model.load_state_dict(torch.load('VAE.pt',map_location=DEVICE2))

<All keys matched successfully>

In [17]:
model.eval()
# trace model with a dummy input
traced_model = torch.jit.trace(model, torch.randn(1, 3, 180, 180))
traced_model.save('VAE_jit2.pt')

VAE(
  (encoder): Sequential(
    (0): Conv2d(3, 6, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(6, 6, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (4): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(6, 6, kernel_size=(5, 5), stride=(1, 1))
    (7): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
    (10): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (decoder): Sequential(
    (0): ConvTranspose2d(8, 6, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(6, 6, kernel_size=(5, 5), stride=(1

In [ ]:
# import torch.onnx

# # Input to the model
# x = torch.randn(1, 3, 180, 180, requires_grad=True)
# torch_out = model(x)

# # Export the model
# torch.onnx.export(model,               # model being run
#                   x,                         # model input (or a tuple for multiple inputs)
#                   "VAE.onnx",   # where to save the model (can be a file or file-like object)
#                   export_params=True,        # store the trained parameter weights inside the model file
#                   opset_version=12,          # the ONNX version to export the model to
#                   do_constant_folding=True,  # whether to execute constant folding for optimization
#                   input_names = ['input'],   # the model's input names
#                   output_names = ['output'], # the model's output names
#                   dynamic_axes={'input' : {0 : 'batch_size'},    # variable lenght axes
#                                 'output' : {0 : 'batch_size'}})

In [27]:
model.save("VAE_jit2.pt" )
torch.jit.save()


TypeError: ignored

In [7]:
model = torch.jit.load("VAE_jit2.pt")

In [15]:
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from torchvision import datasets, transforms
from torchvision.utils import save_image

In [9]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(mean=(0.5, 0.5, 0.5),
                                std=(0.5, 0.5, 0.5)) ])

In [10]:
dataset_test = datasets.ImageFolder('data/test/', transform=transform)
test_loader = DataLoader(dataset=dataset_test, batch_size=16, shuffle=True, drop_last=True, num_workers=4, pin_memory=True)

In [57]:

def generate_reconstructions():
    model.eval()
    x, _ = test_loader.__iter__().next() 

    # x = x.to(DEVICE)
    # x = x[:16].to(DEVICE)
    out, kl_div = model(x)
    x = (x.data + 1) / 2
    out =  (out.data + 1) / 2
    save_image(
        out,
        'samples/Reconstruction.png',
        nrow=8
    )
    save_image(
        x,
        'samples/sample_input.png',
        nrow=8
    )
generate_reconstructions()